<a href="https://colab.research.google.com/github/gut-puncture/AI_code_practice/blob/main/CLM_Self_Coded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Plan
#1. download the model in drive.- DONE
#2. Create a way for the model to be run on the t4 GPU after getting weights from drive. - DONE
#3. Figure out how to get or generate embeddings for all sentences again. - DONE
#4. Get the sentence embeddings in Drive. -DONE
#5. Get dependencies for the reward model calculation
#6. Download dependencies for reward modle calculation
#7. Create alphazero training code:
#    extract model probability distribution
#    Create code for the moves selected
#    input parameters to model
#    Output parameters from model and the moves being played
#8. Reward function gradient descent for both models
#9. Training to happen while counting time to disconnect
#10. Saving model weights to drive and retrieving them

# Setup

In [1]:
# 1️⃣ Mount your Drive so Colab sees it as a local folder.
from google.colab import drive
drive.mount('/content/drive')
# 2️⃣ Define where you want to store the model weights *permanently*.
MODEL_DIR = "/content/drive/MyDrive/phi3_3.8B"


Mounted at /content/drive


In [2]:
# 3️⃣ Install the libraries we'll need.
!pip install --upgrade "transformers==4.41.2" "huggingface_hub>=0.23.0" "accelerate>=0.29.0" sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.3/509.3 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [3]:
# 4️⃣ (ONE-OFF) pull the checkpoint; takes ~8 GB & ~15 minutes on Colab.
from huggingface_hub import login, snapshot_download

login()                                  # Pops an input box. Paste your HF *Access Token* (Settings → Access Tokens → New Token √read).
snapshot_download(
    repo_id="microsoft/phi-3-mini-4k-instruct",   # Exact repo name.
    local_dir=MODEL_DIR,                          # Saves to the Drive folder we chose.
    resume_download=True,                         # Picks up where it left off if Colab disconnects mid-download.
    max_workers=8                                 # Parallel threads; speeds things up a bit.
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

...

sample_finetune.py:   0%|          | 0.00/6.40k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

'/content/drive/MyDrive/phi3_3.8B'

In [9]:
# 5️⃣ Load tokenizer + model and push them onto the GPU automatically.
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype="auto",                 # Uses float16 on GPU, float32 on CPU.
    device_map="auto"                   # transformers + accelerate decide the best device.
)

# 6️⃣ Do a tiny generation just to prove it works.
prompt = "What is the meaning of life? "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

What is the meaning of life? 

# Answer
The meaning of life is a philosophical question concerning the significance of life or existence in general. It has been a topic of debate and contemplation across various cultures and philosophical traditions. Different people and belief systems offer various answers, ranging from religious and spiritual interpretations to existential and humanistic perspectives.

From a religious standpoint, the meaning of life may be understood as fulfilling a divine purpose or adhering to the teachings of a particular faith. For example, in Christianity, the meaning of life is often associated with living in accordance with God's will and seeking eternal life through faith in Jesus Christ.

In existential philosophy, thinkers like Jean-Paul Sartre and Albert Camus have explored the idea that life inherently has no predetermined meaning, and it is up to each individual to create their own purpose through their choices and actions.




In [14]:
prompt = "What is the meaning of life? "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

What is the meaning of life? 

# Answer
The meaning of life is a philosophical question concerning the significance of life or existence in general. It has been a topic of debate and contemplation across various cultures and philosophical traditions. Different people and belief systems offer various answers, ranging from religious and spiritual interpretations to existential and humanistic perspectives.

From a religious standpoint, the meaning of life may be understood as fulfilling a divine purpose or adhering to the teachings of a particular faith. For example, in Christianity, the meaning of life is often associated with living in accordance with God's will and seeking eternal life through faith in Jesus Christ.

In existential philosophy, thinkers like Jean-Paul Sartre and Albert Camus have explored the idea that life inherently has no predetermined meaning, and it is up to each individual to create their own purpose through their choices and actions.




In [13]:
# Correct byte-count for everything under MODEL_DIR
import os, math

total_bytes = 0
for dirpath, dirnames, filenames in os.walk(MODEL_DIR):
    for fname in filenames:                               # <- loop over **files**
        fp = os.path.join(dirpath, fname)                 # absolute path to that file
        total_bytes += os.path.getsize(fp)                # add its byte size

size_gb = total_bytes / 1024**3                           # convert bytes → gibibytes
print(f"We stored {size_gb:.2f} GB of weights in Google Drive at {MODEL_DIR}")

We stored 7.12 GB of weights in Google Drive at /content/drive/MyDrive/phi3_3.8B


# Alphazero-style Scaffolding

## Reward function

In [3]:
!pip install -q bitsandbytes==0.43.1 evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.9 MB/s eta 0:00:00


In [4]:
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig)

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_DIR,
    torch_dtype="auto",
    device_map="auto"                   # transformers + accelerate decide the best device.
)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
prompt = "What is the meaning of life? "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print (outputs.logits[:, -1, :])



In [ ]:
def coherence()

def reward_function(input_ids, logits):


In [ ]:
import torch, math

def token_probs(input_ids):
    """Return list of (entropy, logit_gap, token_margin) per generated token."""
    entropies, gaps, margins = [], [], []
    with torch.no_grad():
        outputs = model(input_ids[:, :-1])  # logits for every position except last
    logits = outputs.logits                # (1, seq_len-1, vocab)
    for step in range(logits.size(1)):
        step_logits = logits[0, step]
        probs = torch.softmax(step_logits, dim=-1)
        H = -(probs * torch.log(probs + 1e-8)).sum().item()             # entropy
        top2 = torch.topk(step_logits, 2).values
        gap = (top2[0] - top2[1]).item()                                # logit gap
        margin = (torch.softmax(top2, 0)[0] - torch.softmax(top2, 0)[1]).item()
        entropies.append(H)
        gaps.append(gap)
        margins.append(margin)
    return entropies, gaps, margins

# Example on the generated sample above
ids = tokenizer(sample, return_tensors="pt").input_ids.to(model.device)
H, gap, margin = token_probs(ids)
print(f"First 5 entropies: {H[:5]}")